# installations & imports

In [81]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

In [82]:
!pip install stable-baselines3[extra]==1.8.0
!pip install huggingface_sb3
!pip install panda_gym==2.0.0
!pip install pyglet==1.2.4

  Using cached stable_baselines3-1.8.0-py3-none-any.whl (174 kB)
  Using cached gym-0.21.0.tar.gz (1.5 MB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [83]:
# !pip install pybullet
# !pip install huggingface_sb3
# !pip install stable_baselines3
# !pip install pyvirtualdisplay
# !pip install 'shimmy>=0.2.1'

In [84]:
import pybullet_envs
# import panda_gym
import gym

import os

from huggingface_sb3 import load_from_hub, package_to_hub

from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env

from huggingface_hub import notebook_login

from pyvirtualdisplay import Display

# create virtual display
source of section: https://huggingface.co/learn/deep-rl-course/unit6/hands-on?fw=pt

In [85]:
virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

# Environment 1: walk
source of section: https://huggingface.co/learn/deep-rl-course/unit6/hands-on?fw=pt

In [86]:
env_id = "AntBulletEnv-v0"
# Create the env
env = gym.make(env_id)

# Get the state space and action space
s_size = env.observation_space.shape[0]
a_size = env.action_space

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


##observation space
29-dimensional array where each value is normalized [0,1].

Order:
<br>0-2: Torso position --> (x,y,z)
<br>3-6: Orientation --> (x,y,z,w)
<br>7-8: Front left leg joint angles (JA) --> (Hip JA), (Ankle JA)
<br>9-10: Back left leg JAs --> (Hip JA), (Ankle JA)
<br>11-12: Back right leg JAs --> (Hip JA), (Ankle JA)
<br>13-14: Front right leg JAs --> (Hip JA), (Ankle JA)
<br>15-17: Torso velocity --> (x,y,z)
<br>18-20: Torso angular velocity --> (x,y,z)
<br>21-22: Front left leg velocity --> (Hip velocity), (Ankle velocity)
<br>23-24: Back left leg velocity --> (Hip velocity), (Ankle velocity)
<br>25-26: Back right leg velocity --> (Hip velocity), (Ankle velocity)
<br>27-28: Front right leg velocity --> (Hip velocity), (Ankle velocity)

In [87]:
print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample())  # Get a random observation

_____OBSERVATION SPACE_____ 

The State Space is:  28
Sample observation [ 0.37050894 -0.03730528 -0.13321948 -0.37220585  1.4687458   1.0308281
  0.04745021  0.47300902 -0.6185841   0.72387993 -1.027576   -0.68009394
 -0.8623388   0.2359985  -0.2915805  -1.5311416  -0.73436016 -1.218651
 -1.886577    0.47775495  0.5973822   0.25681427  0.20584969 -0.22997029
 -1.028546   -0.8852916   0.26647997  1.5997788 ]


## action space
8 possible actions

array:
<br>0: Front left hip
<br>1: Front left ankle
<br>2: Back left hip
<br>3: Back left ankle
<br>4: Bck right hip
<br>5: Back right ankle
<br>6: Front right hip
<br>7: Front right ankle

In [88]:
print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample())  # Take a random action


 _____ACTION SPACE_____ 

The Action Space is:  Box(-1.0, 1.0, (8,), float32)
Action Space Sample [ 0.9722873  -0.5530249   0.6941241  -0.432405   -0.68280655 -0.9262593
 -0.05621326 -0.6343958 ]


## Normalization

In [89]:
# env = make_vec_env(env_id, n_envs=4)

# env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.0)

# A2C model
source of section: https://huggingface.co/learn/deep-rl-course/unit6/hands-on?fw=pt

In [90]:
model = A2C(
    policy="MlpPolicy",
    env=env,
    gae_lambda=0.9,
    gamma=0.99,
    learning_rate=0.00096,
    max_grad_norm=0.5,
    n_steps=8,
    vf_coef=0.4,
    ent_coef=0.0,
    policy_kwargs=dict(log_std_init=-2, ortho_init=False),
    normalize_advantage=False,
    use_rms_prop=True,
    use_sde=True,
    verbose=1,
)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [91]:
model.learn(2_000_000)

/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:190: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `return_info` to return information from the environment resetting.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:195: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the envir

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 400      |
|    ep_rew_mean        | 164      |
| time/                 |          |
|    fps                | 297      |
|    iterations         | 100      |
|    time_elapsed       | 2        |
|    total_timesteps    | 800      |
| train/                |          |
|    entropy_loss       | -2.45    |
|    explained_variance | -0.00402 |
|    learning_rate      | 0.00096  |
|    n_updates          | 99       |
|    policy_loss        | 0.406    |
|    std                | 0.132    |
|    value_loss         | 0.0524   |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 494      |
|    ep_rew_mean        | 179      |
| time/                 |          |
|    fps                | 246      |
|    iterations         | 200      |
|    time_elapsed       | 6        |
|    total_timesteps    | 1600     |
|

KeyboardInterrupt: ignored

In [92]:
# Save the model and  VecNormalize statistics when saving the agent
model.save("a2c-AntBulletEnv-v0")
# env.save("vec_normalize.pkl")

In [93]:
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# Load the saved statistics
eval_env = DummyVecEnv([lambda: gym.make("AntBulletEnv-v0")])
# eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)

#  do not update them at test time
eval_env.training = False
# reward normalization is not needed at test time
eval_env.norm_reward = False

# Load the agent
model = A2C.load("a2c-AntBulletEnv-v0")

mean_reward, std_reward = evaluate_policy(model, env)

print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean reward = 358.26 +/- 67.82


# Visualization
section code from: https://huggingface.co/learn/deep-rl-course/unit6/hands-on?fw=pt

In [94]:
notebook_login()
!git config --global credential.helper store

In [95]:
# push model to hub
package_to_hub(
    model=model,
    model_name=f"a2c-{env_id}",
    model_architecture="A2C",
    env_id=env_id,
    eval_env=eval_env,
    repo_id=f"JJLugeder/a2c-{env_id}",  # Change the username
    commit_message="Initial commit",
)

ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.


/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:440: UserWarning: WARN: The `registry.env_specs` property along with `EnvSpecTree` is deprecated. Please use `registry` directly as a dictionary instead.
  logger.warn(


AssertionError: ignored